In [ ]:
import pandas as pd

df = pd.read_csv(r"C:\Users\compro\week3\자연어\네이버 기사\형일\클랜징\Naver_cleansed_8S.csv")
df = df.dropna()
df

In [ ]:
asdf = df.sample(frac=0.05, random_state=42)  # 5% 데이터만 사용
asdf

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from konlpy.tag import Mecab

m = Mecab()
m_1 = m.nouns(asdf['content'].iloc[0])
m_2 = m.nouns(asdf['content'].iloc[1])

m_1_str = ' '.join(m_1)
m_2_str = ' '.join(m_2)

count_vect = CountVectorizer(min_df=0.05, ngram_range=(1, 2))
a = count_vect.fit_transform([m_1_str, m_2_str]).toarray()

vector_a = a[0]
vector_b = a[1]

def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)
    return dot_product / (norm_A * norm_B)

result = cosine_similarity(vector_a, vector_b)
print("Cosine Similarity:", result)

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from konlpy.tag import Mecab
import numpy as np

m = Mecab()
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')
model = BertModel.from_pretrained('klue/bert-base')

def embed_sentence(sentence, tokenizer, model):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

nouns_list = [m.nouns(content) for content in asdf['content']]
nouns_str_list = [' '.join(nouns) for nouns in nouns_list]

sentence_embeddings = [embed_sentence(sentence, tokenizer, model) for sentence in nouns_str_list]
sentence_embeddings = np.array(sentence_embeddings)

def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)
    return dot_product / (norm_A * norm_B)

# 첫 번째 문장 기준으로 모든 문장의 content 유사도 계산
similarities = []
vector_0 = sentence_embeddings[0]

for i in range(len(sentence_embeddings)):
    vector_i = sentence_embeddings[i]
    sim = cosine_similarity(vector_0, vector_i)
    similarities.append(sim)

sorted_indices = np.argsort(similarities)[::-1]

# 유사도 기반 추천 함수
def find_sim_content_with_similarity(df, sorted_indices, similarities, top_n=10):
    top_indices = sorted_indices[1:top_n+1]
    sim_contents = df.iloc[top_indices].copy()
    sim_contents['similarity'] = [similarities[i] for i in top_indices]
    
    return sim_contents

# 추천 콘텐츠 찾기 (유사도 함께 표시)
similar_contents_with_similarity = find_sim_content_with_similarity(asdf, sorted_indices, similarities, top_n=10)
print(similar_contents_with_similarity[['content', 'similarity']])
